In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from numpy import std
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from math import sqrt

from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score
from time import time

from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
dataset_dir = 'Dataset_Final/Dataset_unido.csv' #Se carga el dataset
df = pd.read_csv(dataset_dir, sep=(',')) #Se lee el archivo csv el cual está separado por comas
df #Se imprime el dataset

,CodDepartamento,Ano,Trimestre,PorcentajeDesempleo,Temperatura,Precipitacion,PIB,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,PorcentajeVacunacion,Vulnerabilidad_numero,Incidencia
0,5,2020,1,12.9,24.149651,7.957837,46018.0,-12.489606,-8.493907,-11.392832,-9.280287,-5.809319,4.227240,0.00,2.0,0.000024
1,5,2020,2,25.2,22.286968,5.173339,46018.0,-67.037634,-44.896057,-55.349462,-62.886022,-50.243011,25.524731,0.00,2.0,0.000825
2,5,2020,3,20.0,22.739094,7.672982,46018.0,-50.040860,-28.913620,-39.594624,-51.555556,-34.839427,17.956272,0.00,2.0,0.026520
3,5,2020,4,15.2,23.157370,8.176094,46018.0,-28.945878,-3.579928,-27.370968,-28.567742,-19.315054,10.191756,0.00,2.0,0.033072
4,5,2021,1,17.9,22.285123,6.276447,52347.0,-32.214670,-0.670507,-31.100230,-34.711982,-20.892857,9.510369,0.66,2.0,0.024737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,88,2020,4,25.3,25.658387,0.000000,1027.0,-57.937993,-29.239427,-60.999642,-41.625090,-32.145878,13.854480,0.00,1.0,0.014034
188,88,2021,1,16.0,26.667411,0.000000,1300.0,-36.820661,-11.887865,-48.219662,-15.671659,-21.868280,9.530722,0.51,1.0,0.005350
189,88,2021,2,16.0,28.111204,0.000000,1300.0,-23.432616,-3.455914,-47.115412,-0.963082,-16.767384,11.880645,13.53,1.0,0.061390
190,88,2021,3,11.9,23.849370,0.000000,1300.0,1.812545,11.936918,-38.234050,31.749821,-7.212903,6.475986,56.77,1.0,0.006413


In [3]:
y = df["Incidencia"]
X = df.iloc[:,:-1]
X

,CodDepartamento,Ano,Trimestre,PorcentajeDesempleo,Temperatura,Precipitacion,PIB,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,PorcentajeVacunacion,Vulnerabilidad_numero
0,5,2020,1,12.9,24.149651,7.957837,46018.0,-12.489606,-8.493907,-11.392832,-9.280287,-5.809319,4.227240,0.00,2.0
1,5,2020,2,25.2,22.286968,5.173339,46018.0,-67.037634,-44.896057,-55.349462,-62.886022,-50.243011,25.524731,0.00,2.0
2,5,2020,3,20.0,22.739094,7.672982,46018.0,-50.040860,-28.913620,-39.594624,-51.555556,-34.839427,17.956272,0.00,2.0
3,5,2020,4,15.2,23.157370,8.176094,46018.0,-28.945878,-3.579928,-27.370968,-28.567742,-19.315054,10.191756,0.00,2.0
4,5,2021,1,17.9,22.285123,6.276447,52347.0,-32.214670,-0.670507,-31.100230,-34.711982,-20.892857,9.510369,0.66,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,88,2020,4,25.3,25.658387,0.000000,1027.0,-57.937993,-29.239427,-60.999642,-41.625090,-32.145878,13.854480,0.00,1.0
188,88,2021,1,16.0,26.667411,0.000000,1300.0,-36.820661,-11.887865,-48.219662,-15.671659,-21.868280,9.530722,0.51,1.0
189,88,2021,2,16.0,28.111204,0.000000,1300.0,-23.432616,-3.455914,-47.115412,-0.963082,-16.767384,11.880645,13.53,1.0
190,88,2021,3,11.9,23.849370,0.000000,1300.0,1.812545,11.936918,-38.234050,31.749821,-7.212903,6.475986,56.77,1.0


#Perform Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [4]:
# División de los datos en train y test
# ==============================================================================
X_train, X_test, Y_train, Y_test = train_test_split(
                                        df.drop(columns = "Incidencia"),
                                        df['Incidencia'],
                                        test_size = 0.3 ,
                                        random_state = 117
                                    )


In [5]:
regressors = [
    LinearRegression(),
    DecisionTreeRegressor(max_depth=7,random_state=329),
    KNeighborsRegressor(),
    SVR(),
    RandomForestRegressor(random_state=329),
    GradientBoostingRegressor(random_state=329),
    ExtraTreesRegressor(random_state=329),
    AdaBoostRegressor(random_state=329)
]

In [6]:
head = 8
for model in regressors[:head]:
    start = time()
    model.fit(X_train, Y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    # prepare the cross-validation procedure
    cv = KFold(n_splits=5)

    #evaluate model
    scores = cross_val_score(model, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)
    scores_r = cross_val_score(model, X_train, Y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(Y_test, y_pred))
    print("\tRMSE:", np.sqrt(mean_squared_error(Y_test, y_pred)))
    print("\tR2 score:", r2_score(Y_test, y_pred))
    print("\tRMSE cv:", np.mean(-1*scores_r))
    print("\tR2 score cv:", np.mean(scores))
    print()

LinearRegression()
	Training time: 0.052s
	Prediction time: 5.036s
	Explained variance: 0.3379820979640996
	RMSE: 0.013631036285613305
	R2 score: 0.3197823865579503
	RMSE cv: 0.014300584321755724
	R2 score cv: 0.21841700728540295

DecisionTreeRegressor(max_depth=7, random_state=329)
	Training time: 0.006s
	Prediction time: 0.210s
	Explained variance: 0.4798074837416624
	RMSE: 0.012042390381554295
	R2 score: 0.46909657265987215
	RMSE cv: 0.01174975482202759
	R2 score cv: 0.45071486695434865

KNeighborsRegressor()
	Training time: 0.005s
	Prediction time: 0.053s
	Explained variance: -0.3138023345540899
	RMSE: 0.01909837924162562
	R2 score: -0.33531322872710745
	RMSE cv: 0.017598878855123606
	R2 score cv: -0.18521806001308308

SVR()
	Training time: 0.003s
	Prediction time: 0.077s
	Explained variance: 0.0
	RMSE: 0.030904503737867302
	R2 score: -2.496501136639113
	RMSE cv: 0.031303076940417715
	R2 score cv: -3.0876696039395606

RandomForestRegressor(random_state=329)
	Training time: 0.266s
	

In [7]:
# División de los datos en train y test
# ==============================================================================
X_train, X_test, Y_train, Y_test = train_test_split(
                                        df.drop(columns = "Incidencia"),
                                        df['Incidencia'],
                                        test_size = 0.2 ,
                                        random_state = 117
                                    )

In [8]:
head = 8
for model in regressors[:head]:
    start = time()
    model.fit(X_train, Y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    # prepare the cross-validation procedure
    cv = KFold(n_splits=5)

    #evaluate model
    scores = cross_val_score(model, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)
    scores_r = cross_val_score(model, X_train, Y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(Y_test, y_pred))
    print("\tRMSE:", np.sqrt(mean_squared_error(Y_test, y_pred)))
    print("\tR2 score:", r2_score(Y_test, y_pred))
    print("\tRMSE cv:", np.mean(-1*scores_r))
    print("\tR2 score cv:", np.mean(scores))
    print()

LinearRegression()
	Training time: 0.009s
	Prediction time: 0.069s
	Explained variance: 0.3612884697146156
	RMSE: 0.013866120614675022
	R2 score: 0.3582447293870542
	RMSE cv: 0.013994777809401582
	R2 score cv: 0.245649050061343

DecisionTreeRegressor(max_depth=7, random_state=329)
	Training time: 0.004s
	Prediction time: 0.063s
	Explained variance: 0.611460769655859
	RMSE: 0.01078957648790785
	R2 score: 0.6114308716255079
	RMSE cv: 0.014407183325880927
	R2 score cv: 0.1667414855741706

KNeighborsRegressor()
	Training time: 0.004s
	Prediction time: 0.065s
	Explained variance: -0.19165201525788778
	RMSE: 0.019019737215518116
	R2 score: -0.2074482362169363
	RMSE cv: 0.01831491375471123
	R2 score cv: -0.2819405149183184

SVR()
	Training time: 0.003s
	Prediction time: 0.051s
	Explained variance: 0.0
	RMSE: 0.03328518820718469
	R2 score: -2.697954224219229
	RMSE cv: 0.030932950732732445
	R2 score cv: -2.707580375328355

RandomForestRegressor(random_state=329)
	Training time: 0.230s
	Predicti

# Prueba DANE

In [9]:
dataset = df.loc[:,['CodDepartamento','Ano','Trimestre','Vulnerabilidad_numero','Incidencia']]
dataset

,CodDepartamento,Ano,Trimestre,Vulnerabilidad_numero,Incidencia
0,5,2020,1,2.0,0.000024
1,5,2020,2,2.0,0.000825
2,5,2020,3,2.0,0.026520
3,5,2020,4,2.0,0.033072
4,5,2021,1,2.0,0.024737
...,...,...,...,...,...
187,88,2020,4,1.0,0.014034
188,88,2021,1,1.0,0.005350
189,88,2021,2,1.0,0.061390
190,88,2021,3,1.0,0.006413


In [10]:
y = dataset["Incidencia"]
X = dataset.iloc[:,:-1]
X

,CodDepartamento,Ano,Trimestre,Vulnerabilidad_numero
0,5,2020,1,2.0
1,5,2020,2,2.0
2,5,2020,3,2.0
3,5,2020,4,2.0
4,5,2021,1,2.0
...,...,...,...,...
187,88,2020,4,1.0
188,88,2021,1,1.0
189,88,2021,2,1.0
190,88,2021,3,1.0


In [11]:
# División de los datos en train y test
# ==============================================================================
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 117)

In [12]:
head = 8
for model in regressors[:head]:
    start = time()
    model.fit(X_train, Y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    # prepare the cross-validation procedure
    cv = KFold(n_splits=5)

    #evaluate model
    scores = cross_val_score(model, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)
    scores_r = cross_val_score(model, X_train, Y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(Y_test, y_pred))
    print("\tRMSE:", np.sqrt(mean_squared_error(Y_test, y_pred)))
    print("\tR2 score:", r2_score(Y_test, y_pred))
    print("\tRMSE cv:", np.mean(-1*scores_r))
    print("\tR2 score cv:", np.mean(scores))
    print()

LinearRegression()
	Training time: 0.006s
	Prediction time: 0.069s
	Explained variance: 0.09065364588854208
	RMSE: 0.016508335348110314
	R2 score: 0.09036730481071598
	RMSE cv: 0.016161611873050884
	R2 score cv: 0.0132045928382859

DecisionTreeRegressor(max_depth=7, random_state=329)
	Training time: 0.005s
	Prediction time: 0.068s
	Explained variance: 0.5170749503938175
	RMSE: 0.012028746431194103
	R2 score: 0.5170521501771455
	RMSE cv: 0.013482880504951814
	R2 score cv: 0.24031147526214633

KNeighborsRegressor()
	Training time: 0.003s
	Prediction time: 0.049s
	Explained variance: -0.2832543637796132
	RMSE: 0.019642517666760933
	R2 score: -0.28781595633420465
	RMSE cv: 0.018145464233140063
	R2 score cv: -0.2562989613217138

SVR()
	Training time: 0.003s
	Prediction time: 0.045s
	Explained variance: 0.0
	RMSE: 0.03328518820718469
	R2 score: -2.697954224219229
	RMSE cv: 0.030932950732732445
	R2 score cv: -2.707580375328355

RandomForestRegressor(random_state=329)
	Training time: 0.179s
	P

In [13]:
# División de los datos en train y test
# ==============================================================================
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 117)

In [14]:
head = 8
for model in regressors[:head]:
    start = time()
    model.fit(X_train, Y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    # prepare the cross-validation procedure
    cv = KFold(n_splits=5)

    #evaluate model
    scores = cross_val_score(model, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)
    scores_r = cross_val_score(model, X_train, Y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(Y_test, y_pred))
    print("\tRMSE:", np.sqrt(mean_squared_error(Y_test, y_pred)))
    print("\tR2 score:", r2_score(Y_test, y_pred))
    print("\tRMSE cv:", np.mean(-1*scores_r))
    print("\tR2 score cv:", np.mean(scores))
    print()

LinearRegression()
	Training time: 0.008s
	Prediction time: 0.071s
	Explained variance: 0.06264724514158859
	RMSE: 0.01625213009680334
	R2 score: 0.033035108941913305
	RMSE cv: 0.016081121367950823
	R2 score cv: 0.04111941694703296

DecisionTreeRegressor(max_depth=7, random_state=329)
	Training time: 0.004s
	Prediction time: 0.050s
	Explained variance: 0.4762450434447919
	RMSE: 0.011979489426997234
	R2 score: 0.4746282183018782
	RMSE cv: 0.011927627934692342
	R2 score cv: 0.40450864618015264

KNeighborsRegressor()
	Training time: 0.003s
	Prediction time: 0.065s
	Explained variance: -0.21146964290358383
	RMSE: 0.01825917166063499
	R2 score: -0.2205407078049726
	RMSE cv: 0.017636664446070777
	R2 score cv: -0.17305704505181926

SVR()
	Training time: 0.003s
	Prediction time: 0.056s
	Explained variance: 0.0
	RMSE: 0.030904503737867302
	R2 score: -2.496501136639113
	RMSE cv: 0.031303076940417715
	R2 score cv: -3.0876696039395606

RandomForestRegressor(random_state=329)
	Training time: 0.198s

In [15]:
# División de los datos en train y test
# ==============================================================================
X_train, X_test, Y_train, Y_test = train_test_split(
                                        df.drop(columns = "Incidencia"),
                                        df['Incidencia'],
                                        test_size = 0.2 ,
                                        random_state = 117
                                    )

In [16]:
from sklearn.model_selection import GridSearchCV

# Decision Tree Regressor

In [17]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              #'learning_rate' : (0.001,0.1,0.05,0.25,0.50,1),
              'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              #'subsample'    : [0.4,0.5,0.6,0.7,0.8,1],
              #'n_estimators' : range(10, 100, 5),
              'max_depth'    : range(1, 20, 1),
              'random_state' : [329]
             }

In [18]:
grid = GridSearchCV(DecisionTreeRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

{'criterion': 'absolute_error', 'max_depth': 4, 'max_features': 'auto', 'random_state': 329} 

DecisionTreeRegressor(criterion='absolute_error', max_depth=4,
                      max_features='auto', random_state=329) 



In [19]:
model_n = model.best_estimator_
model_n

DecisionTreeRegressor(criterion='absolute_error', max_depth=4,
                      max_features='auto', random_state=329)

In [20]:
model = model_n.fit(X_train,Y_train)
predicciones = model_n.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

El error (rmse) de test es:  0.00934089542235024
R2:  0.7087697274078004


In [21]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model_n, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

R2 cv: 0.5885383012228657


In [22]:
sns.distplot(y_pred-Y_test)

ValueError: operands could not be broadcast together with shapes (58,) (39,) 

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

In [ ]:
parameters = {'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              'max_depth'    : range(1, 20, 1),
              'random_state' : [329],
              'ccp_alpha'    : [3.3333333333333333e-06]
             }

In [ ]:
grid = GridSearchCV(DecisionTreeRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.head(3)

# Random Forest Regressor

In [ ]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              #'learning_rate' : (0.001,0.1,0.05,0.25,0.50,1),
              'criterion' : ['squared_error','absolute_error','friedman_mse','poisson'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              #'subsample'    : [0.4,0.5,0.6,0.7,0.8],
              'n_estimators' : range(10, 100, 5),
              'max_depth'    : range(1, 10, 1),
              'random_state' : [117]
             }

In [ ]:
grid = GridSearchCV(RandomForestRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
model_n = model.best_estimator_
model_n

In [ ]:
model = model_n.fit(X_train,Y_train)
predicciones = model_n.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model_n, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores))

In [ ]:
sns.distplot(y_pred-Y_test)

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

# Gradient Boosting

In [ ]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              'learning_rate' : (0.1,0.25,0.50,1),
              #'criterion' : ['friedman_mse', 'mse', 'mae'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              'subsample'    : [0.1,0.2,0.4,0.6,0.8,1],
              'n_estimators' : range(10, 100, 5),
              'max_depth'    : range(1, 10, 1),
              'random_state' : [329]
             }

In [ ]:
grid = GridSearchCV(GradientBoostingRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
model_n = model.best_estimator_
#model_n = GradientBoostingRegressor(max_features='sqrt', n_estimators=70,
#                          random_state=329, subsample=1)
model_n

In [ ]:
model = model_n.fit(X_train,Y_train)
predicciones = model_n.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model_n, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

In [ ]:
sns.distplot(y_pred-Y_test)

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

In [ ]:
mol = GradientBoostingRegressor(max_features='sqrt', n_estimators=29,
                          random_state=329, subsample=0.9)
mol

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(mol, X_test, Y_test, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores))

In [ ]:
mol = mol.fit(X_train,Y_train)
predicciones = mol.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(mol, X_train, Y_train, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores))

#### 

# ExtraTreesRegressor

In [ ]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              #'learning_rate' : (0.001,0.1,0.05,0.25,0.50,1),
              'criterion' : ['squared_error','absolute_error', 'friedman_mse','poisson'],
              'max_features' : ['auto','sqrt', 'log2'],
              #'subsample'    : [0.4,0.5,0.6,0.7,0.8],
              'n_estimators' : range(10, 25, 1),
              'max_depth'    : range(1, 10, 1),
              'random_state' : [117]
             }

In [ ]:
grid = GridSearchCV(ExtraTreesRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
predicciones = model.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

In [ ]:
sns.distplot(y_pred-Y_test)

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

# Random Forest Regressor

In [ ]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              #'learning_rate' : (0.001,0.1,0.05,0.25,0.50,1),
              'criterion' : ['squared_error','absolute_error','friedman_mse','poisson'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              #'subsample'    : [0.4,0.5,0.6,0.7,0.8],
              'n_estimators' : range(10, 25, 1),
              'max_depth'    : range(1, 10, 1),
              'random_state' : [117]
             }

In [ ]:
grid = GridSearchCV(RandomForestRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
predicciones = model.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

In [ ]:
sns.distplot(y_pred-Y_test)

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

# Decision Tree Regressor

In [ ]:
parameters = { #'loss' : ['ls', 'lad', 'huber', 'quantile'],
              #'learning_rate' : (0.001,0.1,0.05,0.25,0.50,1),
              'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'],
              'max_features' : ['auto', 'sqrt', 'log2'],
              #'subsample'    : [0.4,0.5,0.6,0.7,0.8],
              #'n_estimators' : range(10, 25, 1),
              'max_depth'    : range(1, 10, 1),
              'random_state' : [117]
             }

In [ ]:
grid = GridSearchCV(DecisionTreeRegressor(),parameters)
model = grid.fit(X_train,Y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

In [ ]:
predicciones = model.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = Y_test,
        y_pred  = predicciones,
        squared = True
       )

print(f"El error (rmse) de test es: ", np.sqrt(rmse))
print("R2: ",r2_score(Y_test,predicciones))

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=5)

#evaluate model
scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

In [ ]:
sns.distplot(y_pred-Y_test)

In [ ]:
df_1 = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_1.shape

In [ ]:
grid.best_estimator_

In [ ]:
df_1.head(3)

In [ ]:
# Creamos un nuevo objeto de Regresión Lineal
modelo = SVR(kernel = 'rbf', C=100, gamma=0.1, epsilon=0.1)
 
# Entrenamos el modelo, esta vez, con 2 dimensiones
# obtendremos 2 coeficientes, para graficar un plano
modelo.fit(X_train, Y_train)
y_pred = modelo.predict(X_test)
resultado = modelo.score(X_test, Y_test)
# compute the RMSE of our predictions
print("RMSE: ",(np.sqrt(mean_squared_error(Y_test, y_pred))))
print(std(y_pred)) # con esto vamos a ver cual es la precisión de nuestro modelo en porcentaje.
print("R2: ",r2_score(Y_test,y_pred))

In [ ]:
# Creamos un nuevo objeto de Regresión Lineal
modelo = SVR(kernel = 'linear', C=100, gamma="auto")
 
# Entrenamos el modelo, esta vez, con 2 dimensiones
# obtendremos 2 coeficientes, para graficar un plano
modelo.fit(X_train, Y_train)
y_pred = modelo.predict(X_test)
resultado = modelo.score(X_test, Y_test)
# compute the RMSE of our predictions
print("RMSE: ",(np.sqrt(mean_squared_error(Y_test, y_pred))))
print(std(y_pred)) # con esto vamos a ver cual es la precisión de nuestro modelo en porcentaje.
print("R2: ",r2_score(Y_test,y_pred))

In [ ]:
# Creamos un nuevo objeto de Regresión Lineal
modelo = SVR(kernel = 'poly', C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
 
# Entrenamos el modelo, esta vez, con 2 dimensiones
# obtendremos 2 coeficientes, para graficar un plano
modelo.fit(X_train, Y_train)
y_pred = modelo.predict(X_test)
resultado = modelo.score(X_test, Y_test)
# compute the RMSE of our predictions
print("RMSE: ",(np.sqrt(mean_squared_error(Y_test, y_pred))))
print(std(y_pred)) # con esto vamos a ver cual es la precisión de nuestro modelo en porcentaje.
print("R2: ",r2_score(Y_test,y_pred))

In [ ]:
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=1)

#evaluate model
scores = cross_val_score(modelo, X, y, scoring='r2', cv=cv, n_jobs=-1)

# Acá indicamos cuantos fold queremos.
print("R2 cv:", np.mean(scores)) 

## 